In [ ]:
!pip -q install gradio PyPDF2 spacy langchain qdrant-client langchain-huggingface langchain_community anyio
!pip -q install tts ffmpeg-python gdown
!pip -q install opencv-python-headless
!apt-get install ffmpeg
!python -m spacy download en_core_web_sm
!pip -q install gtts playsound
!pip -q install langchain_groq
!pip -q install librosa==0.8.0
!pip install ffmpeg
!pip install librosa
!pip install numpy
!pip install torch
!pip install torchvision
!pip install moviepy
!pip install gdown
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.5

In [ ]:
!git clone https://github.com/Rudrabha/Wav2Lip.git

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 381, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 381 (delta 0), reused 1 (delta 0), pack-reused 378
Receiving objects: 100% (381/381), 534.01 KiB | 6.85 MiB/s, done.
Resolving deltas: 100% (210/210), done.


In [ ]:
pip install -r /content/Wav2Lip/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 53.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.0.25 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84)
ERROR: No matching distribution found for opencv-python==4.1.0.25


In [ ]:
import os
import gdown
import gradio as gr
from PyPDF2 import PdfReader
import os
import subprocess
import uuid
import spacy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
import numpy as np
from langchain_groq import ChatGroq
from gtts import gTTS
import urllib.request

def download_file_from_google_drive(file_url, output_path):
    gdown.download(file_url, output_path, quiet=False)
    print(f"Downloaded {file_url} to {output_path}")

# Create checkpoints directory if it doesn't exist
checkpoints_dir = '/content/Wav2Lip/checkpoints'
os.makedirs(checkpoints_dir, exist_ok=True)

# Google Drive URL and output file path
google_drive_url = "https://drive.google.com/uc?id=1l18x5wRD8numA1heqjkAN8sGRmnMADrS"
output_file_path = os.path.join(checkpoints_dir, 'wav2lip.pth')

# Download the file
download_file_from_google_drive(google_drive_url, output_file_path)


Downloading...
From (original): https://drive.google.com/uc?id=1l18x5wRD8numA1heqjkAN8sGRmnMADrS
From (redirected): https://drive.google.com/uc?id=1l18x5wRD8numA1heqjkAN8sGRmnMADrS&confirm=t&uuid=5c9835c6-a6af-4d4e-93a9-a686924f5631
To: /content/Wav2Lip/checkpoints/wav2lip.pth
100%|██████████| 436M/436M [00:07<00:00, 61.7MB/s]

Downloaded https://drive.google.com/uc?id=1l18x5wRD8numA1heqjkAN8sGRmnMADrS to /content/Wav2Lip/checkpoints/wav2lip.pth


In [ ]:

# Initialize the groq API key
groq_api_key = "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"  # Replace with actual method to fetch the key

# Initialize LLM
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key)

# Custom VectorStoreRetriever class
class VectorStoreRetriever:
    def __init__(self, client, collection_name, embedding_function):
        self.client = client
        self.collection_name = collection_name
        self.embedding_function = embedding_function

    def retrieve(self, query):
        embedding = self.embedding_function.embed_documents([query])[0]
        search_result = self.client.search(
            collection_name=self.collection_name,
            query_vector=embedding,
            limit=5  # Adjust as needed
        )
        return [hit.payload['content'] for hit in search_result]

try:
    import numpy as np
    np_version = np.__version__
    print(f"Using numpy version {np_version}")

    nlp = spacy.load('en_core_web_sm')

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False
    )

    embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

    sample_text = "This is a sample text to determine the embedding dimension."
    sample_embedding = embedding_function.embed_documents([sample_text])[0]
    embedding_dimension = len(sample_embedding)

    client = QdrantClient(":memory:")
    collection_name = "cv_sections"

    if not client.collection_exists(collection_name):
        client.recreate_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(
                size=embedding_dimension,
                distance=models.Distance.COSINE,
            ),
        )

    def extract_cv_info(cv_text):
        doc = nlp(cv_text)
        sections = {"skills": [], "experience": [], "education": []}
        current_section = None
        for token in doc:
            if token.text.lower() in ["skills", "experience", "education"]:
                current_section = token.text.lower()
            elif current_section:
                sections[current_section].append(token.text)
        for key in sections:
            sections[key] = " ".join(sections[key])
        return sections

    def chunk_text(text):
        return text_splitter.split_text(text)

    def generate_embeddings(text_chunks):
        return embedding_function.embed_documents(text_chunks)

    def index_data_in_qdrant(data, collection_name):
        points = []
        for section, content in data.items():
            chunks = chunk_text(content)
            embeddings = generate_embeddings(chunks)
            for i, embedding in enumerate(embeddings):
                points.append(models.PointStruct(
                    id=str(uuid.uuid4()),
                    vector=embedding,
                    payload={"section": section, "content": chunks[i]}
                ))
        client.upsert(collection_name=collection_name, points=points)

    def retrieve_data_with_filter(collection_name, filter_conditions):
        query_filter = models.Filter(
            must=[models.FieldCondition(
                key=key,
                match=models.MatchValue(value=value)
            ) for key, value in filter_conditions.items()]
        )
        query_vector = np.zeros(embedding_dimension, dtype=np.float32)
        search_result = client.search(
            collection_name=collection_name,
            query_vector=query_vector.tolist(),
            limit=10,
            query_filter=query_filter
        )
        return [hit.payload for hit in search_result]

    def extract_text_from_pdf(pdf_file):
        text = ""
        pdf_reader = PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

    def convert_cv_to_structured_docs(cv_text, collection_name, filter_conditions):
        structured_data = extract_cv_info(cv_text)
        index_data_in_qdrant(structured_data, collection_name)
        return retrieve_data_with_filter(collection_name, filter_conditions)

    template = """Answer the following question from the context
    context = {context}
    question = {question}
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)

    def get_context(query, retriever):
        return retriever.retrieve(query)

    def respond_to_query(query, retriever):
        context = get_context(query, retriever)
        response = llm.invoke(prompt.format(question=query, context=" ".join(context)))
        return response





    def create_avatar_video(audio_file_path):
        wav2lip_dir = '/content/Wav2Lip'
        os.chdir(wav2lip_dir)

        # Define paths
        checkpoint_path = '/content/Wav2Lip/checkpoints/wav2lip.pth'
        face_path = '/content/3249935-uhd_3840_2160_25fps.mp4'
        result_path = 'results/result_voice.mp4'

        # Ensure face video exists
        if not os.path.exists(face_path):
            print(f"Face video does not exist: {face_path}")
            return None

        # Ensure audio file exists
        if not os.path.exists(audio_file_path):
            print(f"Audio file does not exist: {audio_file_path}")
            return None

        # Ensure results directory exists
        os.makedirs(os.path.dirname(result_path), exist_ok=True)

        command = [
            'python', 'inference.py',
            '--checkpoint_path', checkpoint_path,
            '--face', face_path,
            '--audio', audio_file_path,
            '--nosmooth'
        ]

        try:
            print(f"Running Wav2Lip inference with command: {' '.join(command)}")
            result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("Avatar video generation command executed successfully")
            print(f"Command output: {result.stdout}")
            print(f"Command error output: {result.stderr}")

            # Check if the video file is created
            if os.path.exists(result_path):
                print("Video file created successfully")
                with open(result_path, "rb") as video_file:
                    return video_file.read()
            else:
                print("Video file was not created.")
                return None
        except subprocess.CalledProcessError as e:
            print(f"Wav2Lip inference error: {e.stderr}")
            print(f"Command output: {e.stdout}")
            return None
        except Exception as e:
            print(f"Unexpected error during video creation: {str(e)}")
            return None

    def text_to_speech(text):
        from gtts import gTTS
        import os
        import subprocess

        language = 'en'
        tts = gTTS(text=text, lang=language, slow=False)
        audio_file = "response.mp3"
        tts.save(audio_file)

        # Convert MP3 to WAV for Wav2Lip compatibility
        wav_output = "temp/temp.wav"

        # Ensure the temp directory exists
        os.makedirs(os.path.dirname(wav_output), exist_ok=True)

        # Remove the existing file if it exists
        if os.path.exists(wav_output):
            os.remove(wav_output)

        ffmpeg_command = [
            'ffmpeg', '-y', '-i', audio_file, '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '24000', wav_output
        ]

        try:
            result = subprocess.run(ffmpeg_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("FFmpeg conversion successful")
            print(f"FFmpeg command output: {result.stdout}")
            if os.path.exists(wav_output):
                print("WAV file created successfully")
                return wav_output
            else:
                print("WAV file was not created.")
                return None
        except subprocess.CalledProcessError as e:
            print(f"FFmpeg conversion error: {e.stderr}")
            print(f"FFmpeg command output: {e.stdout}")
            return None
        except Exception as e:
            print(f"Unexpected error during audio conversion: {str(e)}")
            return None

    retriever = VectorStoreRetriever(client=client, collection_name=collection_name, embedding_function=embedding_function)

    def chatbot(cv_file, query):
        cv_text = extract_text_from_pdf(cv_file)
        filter_conditions = {}  # Define appropriate filter conditions based on requirements
        structured_docs = convert_cv_to_structured_docs(cv_text, collection_name, filter_conditions)
        response = respond_to_query(query, retriever)
        response_text = response.content  # Extract text content from AIMessage object
        audio_file_path = text_to_speech(response_text)
        video_file_path = create_avatar_video(audio_file_path)
        return response_text, audio_file_path, video_file_path

    inputs = [
        gr.File(label="Upload CV (PDF)"),
        gr.Textbox(label="Enter your query")
    ]

    outputs = [
        gr.Textbox(label="Response"),
        gr.Audio(label="Audio Response"),
        gr.Video(label="Avatar Video Response")
    ]

    gr.Interface(fn=chatbot, inputs=inputs, outputs=outputs, title="CV Chatbot").launch(debug=True)

except Exception as e:
    print(f"An error occurred: {str(e)}")



Using numpy version 1.25.2


Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7d6173a94d30>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 1175, in _make_controller_from_path
    lib_controller = controller_class(
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-5007b62f.3.23.dev.so: cannot open shared object file: No such file or directory


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

<ipython-input-5-f6897f0d37eb>:47: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4f00e3f6ced12d036b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
